In [2]:
!pip install openai psycopg2-binary pandas sqlalchemy gradio

Defaulting to user installation because normal site-packages is not writeable
  Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl.metadata (4.1 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ---------------------------------------- 0.0/54.2 MB ? eta -:--:--
    --------------------------------------- 1.3/54.2 MB 7.6 MB/s eta 0:00:07
   - -------------------------------------- 2.6/54.2 MB 6.9 MB/s eta 0:00:08
   --- ------------------------------------ 4.5/54.2 MB 7.5 MB/s eta 0:00:07
   ---- ----------------------------------- 6.3/54.2 MB 7.7 MB/s eta 0:00:07
   ----- ---------------------------------- 7.9/54.2 MB 7.7 MB/s eta 0:00:06
   ------ --------------------------------- 9.4/54.2 MB 7.7 MB/s eta 0:00:06
   -------- ------------------------------- 11.3/54.2 MB 7.8 MB/s eta 0:00:06
   --------- ------------------------------ 13.1/54.2 MB 7.9 MB/s eta 0:00:06
   ---------- ------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\timmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import gradio as gr
import openai
import pandas as pd
from sqlalchemy import create_engine

# --- Setup ---
openai_api_key = "Your_openai_api_key"  # (redact your real key before sharing!)
client = openai.OpenAI(api_key=openai_api_key)

PG_USER = "your_username" # e.g., postgres
PG_PWD = "your_password"
PG_HOST = "" # e.g., database-final-project.cz6cu0sc42u6.us-east-1.rds.amazonaws.com
PG_PORT = 5432
PG_DB = "your_db_name" # e.g., postgres

engine = create_engine(f"postgresql+psycopg2://{PG_USER}:{PG_PWD}@{PG_HOST}:{PG_PORT}/{PG_DB}")

# --- Get Column Names ---
df_sample = pd.read_sql_query("SELECT * FROM companies LIMIT 1;", engine)
column_names = df_sample.columns.tolist()

def generate_sql(question):
    prompt = f"""
    You are a helpful assistant for a PostgreSQL company database.
    The table `companies` has columns: {', '.join(column_names)}.
    Question: {question}
    Only output the SQL code.
    """
    response = client.chat.completions.create(
        model="gpt-4o",  # Changed from "gpt-4.1" which doesn't exist
        messages=[{"role": "user", "content": prompt}]
    )
    sql = response.choices[0].message.content.strip()
    if sql.startswith("```"):
        sql = sql.strip("```").strip("sql").strip()
    return sql

def ask_db(question):
    sql = generate_sql(question)
    try:
        df = pd.read_sql_query(sql, engine)
        return df, ""
    except Exception as e:
        return None, f"Error running SQL: {e}\nSQL: {sql}"

with gr.Blocks() as demo:
    gr.Markdown(f"### Companies Table Columns:\n\n**{', '.join(column_names)}**")
    with gr.Row():
        with gr.Column():
            question_input = gr.Textbox(label="Ask a question about the companies table", lines=2)
            submit_btn = gr.Button("Submit")
            output = gr.Dataframe(label="Query Result", visible=True)
            error_output = gr.Textbox(label="Error Message", visible=True)
            
            def on_submit(question):
                df, err = ask_db(question)
                # Return values directly instead of using .update()
                return df, err
            
            # Connect both textbox submit and button click to the same handler
            question_input.submit(
                on_submit,
                inputs=question_input,
                outputs=[output, error_output]
            )
            submit_btn.click(
                on_submit,
                inputs=question_input,
                outputs=[output, error_output]
            )

demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
